# Use of a Neural-Network for Constitutive Law

Import all the useful libraries before first run
We need here the classic ones such as:
- math
- numpy
- pandas
- matplotlib

And for the Neural Network, we also need to import parts of the keras module of TensorFlow

In [1]:
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp,log

## Load the test database

Read the Test database

In [2]:
dataPath = '.'
read = np.load(dataPath + '/DatatestWithDerivatives.npz')
testData = read['testData']
nrows = testData.shape[0]

In [3]:
eps_test = testData[:,0].reshape(nrows,1)
epsp_test = testData[:,1].reshape(nrows,1)
T_test = testData[:,2].reshape(nrows,1)
sig_test = testData[:,3].reshape(nrows,1)

 ## Load the NN parameters

In [4]:
ANN_name = '3-15-7-1-sigmoid'

NN = np.load(dataPath + '/' + ANN_name + '.npz')
print (NN.files)
w1 = NN['w1']
w2 = NN['w2']
w3 = NN['w3']
b1 = NN['b1']
b2 = NN['b2']
b3 = NN['b3']
minEntries = NN['minEntries']
maxEntries = NN['maxEntries']
rangeEntries = maxEntries - minEntries
logBase = NN['logBase']
w1, b1, w2, b2, w3, b3, minEntries, maxEntries, rangeEntries, logBase

['logBase', 'minEntries', 'maxEntries', 'w1', 'b1', 'w2', 'b2', 'w3', 'b3']


(array([[-1.71193019e-01, -4.98235881e-01,  1.57230926e+00],
        [-1.82183892e-01, -2.79642552e-01,  1.38154745e+00],
        [-1.21008851e-01, -9.36221778e-02, -1.28485608e+00],
        [ 6.77952528e-01,  7.85129726e-01, -3.60040736e+00],
        [-8.22493362e+00,  1.02857426e-01, -1.17000186e+00],
        [-2.62212038e-01, -3.70604903e-01,  1.53059375e+00],
        [ 3.63313168e-01,  3.07944298e-01, -1.73980820e+00],
        [-1.71282619e-01, -9.36020970e-01, -7.00896144e-01],
        [-2.93405890e+00,  2.49693431e-02, -8.18501532e-01],
        [-2.98778355e-01, -5.17931700e-01, -7.58085251e+00],
        [ 1.17702794e+00, -3.08994055e-01, -1.68417966e+00],
        [-2.31339216e-01, -2.90423155e-01,  1.50245142e+00],
        [ 2.35784103e+02, -9.92930010e-02,  7.60096788e-01],
        [-2.33444443e+01,  1.64729804e-01, -9.49119210e-01],
        [ 3.29279006e-01,  5.42672016e-02, -1.55521262e+00]], dtype=float32),
 array([[-0.5497108 ],
        [-1.0076679 ],
        [ 1.146429  ],

## Reshape data

In [5]:
epsp_test = np.log(epsp_test / logBase)
inputTest = (np.hstack([eps_test, epsp_test, T_test]) - np.array(minEntries)[0:3]) / np.array(rangeEntries)[0:3]
inputTest

array([[0.64999785, 0.90552119, 0.65628871],
       [0.77199114, 0.8999689 , 0.45541125],
       [0.64433951, 0.99445344, 0.11009941],
       ...,
       [0.75192854, 0.99520977, 0.75841439],
       [0.74276917, 0.88747863, 0.12681504],
       [0.68998248, 0.88707664, 0.5388862 ]])

Computes the target

In [6]:
def tanh2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = np.tanh(l1)
    l2 = w2.dot(f1) + b2
    f2 = np.tanh(l2)
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

In [7]:
def sigmoid2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = 1/(1 + np.exp(-l1))
    l2 = w2.dot(f1) + b2
    f2 = 1/( 1 +np.exp(-l2))
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

Setup data

In [8]:
inp = inputTest.T
sig = sig_test
#inp, sig

Rebuild the real $\sigma$

In [9]:
if ('tanh' in ANN_name) : SigmaNN = tanh2Layers(inp)
elif ('sigmoid' in ANN_name) : SigmaNN = sigmoid2Layers(inp)
else : SigmaNN = 0
SigmaNN

array([[1232.2282292 , 1335.3603057 , 1470.46979899, ..., 1208.21439896,
        1465.07933384, 1287.15287545]])

In [10]:
w1.shape, inp.shape, b1.shape

((15, 3), (3, 5000), (15, 1))

In [11]:
comment = 'C     '
cline = 'C **********************************************************************\n'
block =   '      '

In [14]:
var = "%.12f"

def toFortran(line):
    line = line.replace("+-", "-")
    line = block + line + '\n'
    return line

In [15]:
code = ''
code += cline
code += comment + 'Input data is eps, epsp, temp\n'
code += cline

code += comment + 'xeps = (eps - minEntries[0]) / rangeEntries[0]\n'
line = 'xeps = (eps -'+var%(minEntries[0])+')/'+var%(rangeEntries[0])
#code += block+line+'\n'
code += toFortran(line)
#print(line)

code += comment + 'xepsp = (log(epsp/logBase) - minEntries[1]) / rangeEntries[1]\n'
line = 'xepsp = (log(epsp/'+var%(logBase[0])+') -'+var%(minEntries[1])+')/'+var%(rangeEntries[1])
code += toFortran(line)
#print(line)

code += comment + 'xtemp = (temp - minEntries[2]) / rangeEntries[2]\n'
line = 'xtemp = (temp -'+var%(minEntries[2])+')/'+var%(rangeEntries[2])
code += toFortran(line)
#print(line)

code += comment + 'l1 = w1.dot(inputData) + b1\n'
for i in range(b1.shape[0]):
    line = 'xl1'+str(i)+" = "+var%(w1[i,0]) + '*xeps+' + var%(w1[i,1]) + '*xepsp+' + var%(w1[i,2]) + '*xtemp+' + var%(b1[i,0])
    code += toFortran(line)
    #print(line)
    
code += comment + 'f1 = 1/(1 + np.exp(-l1))\n'
for i in range(b1.shape[0]):
    line = 'xf1'+str(i)+" = 1.0/(1.0 + exp(-xl1"+str(i)+"))"
    code += toFortran(line)
    #print(line)
    
code += comment + 'l2 = w2.dot(f1) + b2\n'
for i in range(b2.shape[0]):
    line = 'xl2'+str(i)+' = '
    for j in range(w2.shape[1]):
        if (j!=0): line +='+'
        line += var%(w2[i,j]) + '*xf1'+str(j)
    line+="+"+var%(b2[i,0])
    code += toFortran(line)
    #print(line)
    
code += comment + 'f2 = 1/(1 + np.exp(-l2))\n'
for i in range(b2.shape[0]):
    line = 'xf2'+str(i)+" = 1.0/(1.0 + exp(-xl2"+str(i)+"))"
    code += toFortran(line)
    #print(line)
    
code += comment + 'xsigP = w3.dot(f2) + b3\n'
line = 'xsigP = '
for j in range(w3.shape[1]):
    if (j!=0): line +='+'
    line += var%(w3[0,j]) + '*xf2'+str(j)
line+="+"+var%(b3[0,0])
code += toFortran(line)
#print(line)

code += comment + 'sigP = xsigP * rangeEntries[3] + minEntries[3]\n'
line = 'sigP = '+var%(rangeEntries[3])+'*xsigP+'+var%(minEntries[3])
code += toFortran(line)
#print(line)

In [16]:
print(code)

C **********************************************************************
C     Input data is eps, epsp, temp
C **********************************************************************
C     xeps = (eps - minEntries[0]) / rangeEntries[0]
      xeps = (eps -0.000000000000)/1.000000000000
C     xepsp = (log(epsp/logBase) - minEntries[1]) / rangeEntries[1]
      xepsp = (log(epsp/1.000000000000) -0.000000000000)/10.819778284410
C     xtemp = (temp - minEntries[2]) / rangeEntries[2]
      xtemp = (temp -20.000000000000)/480.000000000000
C     l1 = w1.dot(inputData) + b1
      xl10 = -0.171193018556*xeps-0.498235881329*xepsp+1.572309255600*xtemp-0.549710810184
      xl11 = -0.182183891535*xeps-0.279642552137*xepsp+1.381547451019*xtemp-1.007667899132
      xl12 = -0.121008850634*xeps-0.093622177839*xepsp-1.284856081009*xtemp+1.146428942680
      xl13 = 0.677952528000*xeps+0.785129725933*xepsp-3.600407361984*xtemp-1.056580066681
      xl14 = -8.224933624268*xeps+0.102857425809*xepsp-1.1700018644

In [23]:
testData[0,:]


array([ 6.49997854e-01,  1.79894090e+04,  3.35018578e+02,  1.23210657e+03,
        1.32070915e+02,  5.60676950e-04, -9.25727265e-01])

In [24]:
testData[0]
eps = testData[0,0]
epsp = testData[0,1]
temp = testData[0,2]
eps,epsp,temp

(0.6499978537536213, 17989.408960982073, 335.01857843791004)

In [25]:
# **********************************************************************
#     Input data is eps, epsp, temp
# **********************************************************************
#     xeps = (eps - minEntries[0]) / rangeEntries[0]
xeps = (eps -0.000000000000)/1.000000000000
#     xepsp = (log(epsp/logBase) - minEntries[1]) / rangeEntries[1]
xepsp = (log(epsp/1.000000000000) -0.000000000000)/10.819778284410
#     xtemp = (temp - minEntries[2]) / rangeEntries[2]
xtemp = (temp -20.000000000000)/480.000000000000
#     l1 = w1.dot(inputData) + b1
xl10 = -0.171193018556*xeps-0.498235881329*xepsp+1.572309255600*xtemp-0.549710810184
xl11 = -0.182183891535*xeps-0.279642552137*xepsp+1.381547451019*xtemp-1.007667899132
xl12 = -0.121008850634*xeps-0.093622177839*xepsp-1.284856081009*xtemp+1.146428942680
xl13 = 0.677952528000*xeps+0.785129725933*xepsp-3.600407361984*xtemp-1.056580066681
xl14 = -8.224933624268*xeps+0.102857425809*xepsp-1.170001864433*xtemp-0.834357798100
xl15 = -0.262212038040*xeps-0.370604902506*xepsp+1.530593752861*xtemp-0.557993113995
xl16 = 0.363313168287*xeps+0.307944297791*xepsp-1.739808201790*xtemp-0.515264987946
xl17 = -0.171282619238*xeps-0.936020970345*xepsp-0.700896143913*xtemp-0.525115251541
xl18 = -2.934058904648*xeps+0.024969343096*xepsp-0.818501532078*xtemp+0.164255663753
xl19 = -0.298778355122*xeps-0.517931699753*xepsp-7.580852508545*xtemp-1.864312648773
xl110 = 1.177027940750*xeps-0.308994054794*xepsp-1.684179663658*xtemp-0.859441757202
xl111 = -0.231339216232*xeps-0.290423154831*xepsp+1.502451419830*xtemp-1.198348641396
xl112 = 235.784103393555*xeps-0.099293000996*xepsp+0.760096788406*xtemp+0.819592595100
xl113 = -23.344444274902*xeps+0.164729803801*xepsp-0.949119210243*xtemp-0.500462353230
xl114 = 0.329279005527*xeps+0.054267201573*xepsp-1.555212616920*xtemp-0.580829739571
#     f1 = 1/(1 + np.exp(-l1))
xf10 = 1.0/(1.0 + exp(-xl10))
xf11 = 1.0/(1.0 + exp(-xl11))
xf12 = 1.0/(1.0 + exp(-xl12))
xf13 = 1.0/(1.0 + exp(-xl13))
xf14 = 1.0/(1.0 + exp(-xl14))
xf15 = 1.0/(1.0 + exp(-xl15))
xf16 = 1.0/(1.0 + exp(-xl16))
xf17 = 1.0/(1.0 + exp(-xl17))
xf18 = 1.0/(1.0 + exp(-xl18))
xf19 = 1.0/(1.0 + exp(-xl19))
xf110 = 1.0/(1.0 + exp(-xl110))
xf111 = 1.0/(1.0 + exp(-xl111))
xf112 = 1.0/(1.0 + exp(-xl112))
xf113 = 1.0/(1.0 + exp(-xl113))
xf114 = 1.0/(1.0 + exp(-xl114))
#     l2 = w2.dot(f1) + b2
xl20 = 0.000755758665*xf10+0.348879784346*xf11-1.622294902802*xf12-1.744126558304*xf13+1.189603447914*xf14+0.773437738419*xf15-1.077992796898*xf16+0.195966541767*xf17+0.371459305286*xf18-2.175866603851*xf19+0.267415910959*xf110+0.117423757911*xf111-1.837396621704*xf112-1.631284832954*xf113-0.915312469006*xf114-0.220450609922
xl21 = -1.520736217499*xf10-0.598750412464*xf11-0.567928969860*xf12+0.733987450600*xf13-1.956816315651*xf14-0.990542054176*xf15-0.024108286947*xf16-0.921862065792*xf17-1.585801959038*xf18-0.359389543533*xf19-0.568457961082*xf110-0.462055325508*xf111+0.937371969223*xf112+0.713293492794*xf113-0.270313948393*xf114-0.580874204636
xl22 = -0.263961493969*xf10+0.340663641691*xf11-1.086898207664*xf12-0.190290063620*xf13+1.526942133904*xf14+0.134074971080*xf15-0.458628684282*xf16-0.210141628981*xf17+0.690639853477*xf18-0.559289455414*xf19-0.561830639839*xf110+0.010643188842*xf111-2.779141187668*xf112+2.203900575638*xf113-0.874182760715*xf114-0.327307760715
xl23 = -1.667145609856*xf10-1.261272549629*xf11-1.568691968918*xf12-1.300865530968*xf13-0.965722560883*xf14-1.076323390007*xf15-0.399949252605*xf16-2.976247072220*xf17-3.980113983154*xf18-1.212576389313*xf19-0.478132873774*xf110-1.447541594505*xf111-0.287288188934*xf112-0.847129225731*xf113-0.899507462978*xf114-0.965191483498
xl24 = -0.421280384064*xf10-0.564329922199*xf11+1.009604096413*xf12+0.335408806801*xf13-0.515781700611*xf14-0.189889833331*xf15+0.561685442924*xf16-0.274590790272*xf17-0.640589475632*xf18-0.454831272364*xf19+0.185221791267*xf110-1.204036474228*xf111+2.715961456299*xf112-2.415366411209*xf113+0.508277475834*xf114+0.001537762932
xl25 = -1.879916071892*xf10-1.522165775299*xf11-0.468759000301*xf12+1.417994260788*xf13+0.370010912418*xf14-2.718472957611*xf15+0.469961762428*xf16-2.314279079437*xf17-0.682480514050*xf18-1.616387128830*xf19+0.202725619078*xf110-1.451031565666*xf111+0.099444560707*xf112-1.293660640717*xf113+0.203722134233*xf114-0.991643249989
xl26 = -0.589749693871*xf10-0.571380496025*xf11+0.120540380478*xf12+0.763658463955*xf13-1.608781814575*xf14-0.913990020752*xf15+0.719076871872*xf16-1.206779360771*xf17-0.747950553894*xf18-0.689776837826*xf19-0.001232341980*xf110-0.592846393585*xf111+1.381900072098*xf112+0.327141195536*xf113+0.125081762671*xf114-0.179510131478
#     f2 = 1/(1 + np.exp(-l2))
xf20 = 1.0/(1.0 + exp(-xl20))
xf21 = 1.0/(1.0 + exp(-xl21))
xf22 = 1.0/(1.0 + exp(-xl22))
xf23 = 1.0/(1.0 + exp(-xl23))
xf24 = 1.0/(1.0 + exp(-xl24))
xf25 = 1.0/(1.0 + exp(-xl25))
xf26 = 1.0/(1.0 + exp(-xl26))
#     xsigP = w3.dot(f2) + b3
xsigP = -1.147175073624*xf20+0.396453082561*xf21-1.731730222702*xf22+0.341490298510*xf23+0.917140424252*xf24+0.416720062494*xf25+0.469032645226*xf26-0.399738907814
#     sigP = xsigP * rangeEntries[3] + minEntries[3]
sigP = 977.555715042962*xsigP+579.184642915415

In [27]:
sigP, testData[0,3]

(1232.228229196241, 1232.1065669321008)

## Dérivation directe

In [28]:
def tanhPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    tanhx = np.tanh(w1.dot(x) + b1)
    p2 = w3v * (1 - np.tanh(w2.dot(tanhx) + b2)**2)
    p3 = (w2.T).dot(p2)
    p5 = p3 * (1 - tanhx**2)
    s = (w1.T).dot(p5)
    return s

In [29]:
def sigmoidPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    expx = np.exp(-(w1.dot(x) + b1))
    exp2 = np.exp(w2.dot(1/(1 + expx)) + b2)
    p1 = w3v*(exp2 / (1 + exp2)**2)
    p2 = expx / (1 + expx)**2
    s = (w1.T).dot((w2.T).dot(p1) * p2)
    return s

In [30]:
code = ''

code += comment + 'xeps = (eps - minEntries[0]) / rangeEntries[0]\n'
line = 'xeps = (eps -'+var%(minEntries[0])+')/'+var%(rangeEntries[0])
#code += block+line+'\n'
code += toFortran(line)
#print(line)

code += comment + 'xepsp = (log(epsp/logBase) - minEntries[1]) / rangeEntries[1]\n'
line = 'xepsp = (log(epsp/'+var%(logBase[0])+') -'+var%(minEntries[1])+')/'+var%(rangeEntries[1])
code += toFortran(line)
#print(line)

code += comment + 'xtemp = (temp - minEntries[2]) / rangeEntries[2]\n'
line = 'xtemp = (temp -'+var%(minEntries[2])+')/'+var%(rangeEntries[2])
code += toFortran(line)
#print(line)

code += comment + 'expx = np.exp(-(w1.dot(x) + b1))\n'
for i in range(b1.shape[0]):
    line = 'expx'+str(i)+" = exp(-("+var%(w1[i,0]) + '*xeps+' + var%(w1[i,1]) + '*xepsp+' + var%(w1[i,2]) + '*xtemp+' + var%(b1[i,0])+'))'
    code += toFortran(line)
    #print(line)
    
code += comment + 'exp2 = np.exp(w2.dot(1/(1 + expx)) + b2)\n'
for i in range(b2.shape[0]):
    line = 'exp2'+str(i)+' = exp('
    for j in range(w2.shape[1]):
        if (j!=0): line +='+'
        line += var%(w2[i,j]) + '*(1/(1+expx'+str(j)+'))'
    line+="+"+var%(b2[i,0])+')'
    code += toFortran(line)
    #print(line)
    
code += comment + 'p1 = w3v*(exp2 / (1 + exp2)**2)\n'
for i in range(b2.shape[0]):
    line = 'p1'+str(i)+' = '+var%(w3[0,i])+'*(exp2'+str(i)+'/(1+exp2'+str(i)+')**2)'
    code += toFortran(line)

code += comment + 'p2 = expx / (1 + expx)**2\n'
for i in range(b1.shape[0]):
    line = 'p2'+str(i)+' = expx'+str(i)+'/(1+expx'+str(i)+')**2'
    code += toFortran(line)

code += comment + 'xs = (w2.T).dot(p1) * p2\n'
for i in range(b1.shape[0]):
    line = 'xs'+str(i)+' = ('
    for j in range(b2.shape[0]):
        if(j>0): line+='+'
        line += var%(w2[j,i])+'*p1'+str(j)
    line +=')*p2'+str(i)
    code += toFortran(line)
    
code += comment + 's = (w1.T).dot(xs)\n'
for i in range(w1.shape[1]):
    line = 's'+str(i)+' = '
    for j in range(w1.shape[0]):
        if (j>0): line +='+'
        line += var%(w1[j,i])+"*xs"+str(j)
    code += toFortran(line)

line = 'deps = s0 * '+var%(rangeEntries[3])+'/'+var%(rangeEntries[0])
code += toFortran(line)
line = 'depsp = s1 * '+var%(rangeEntries[3])+'/(epsp*'+var%(rangeEntries[1])+')'
code += toFortran(line)
line = 'dtemp = s2 * '+var%(rangeEntries[3])+'/'+var%(rangeEntries[2])
code += toFortran(line)

    #scaleOut = np.array([[rangeEntries[3]/rangeEntries[0]],[rangeEntries[3]/rangeEntries[1]],[rangeEntries[3]/rangeEntries[2]]])


In [31]:
print(code)

C     xeps = (eps - minEntries[0]) / rangeEntries[0]
      xeps = (eps -0.000000000000)/1.000000000000
C     xepsp = (log(epsp/logBase) - minEntries[1]) / rangeEntries[1]
      xepsp = (log(epsp/1.000000000000) -0.000000000000)/10.819778284410
C     xtemp = (temp - minEntries[2]) / rangeEntries[2]
      xtemp = (temp -20.000000000000)/480.000000000000
C     expx = np.exp(-(w1.dot(x) + b1))
      expx0 = exp(-(-0.171193018556*xeps-0.498235881329*xepsp+1.572309255600*xtemp-0.549710810184))
      expx1 = exp(-(-0.182183891535*xeps-0.279642552137*xepsp+1.381547451019*xtemp-1.007667899132))
      expx2 = exp(-(-0.121008850634*xeps-0.093622177839*xepsp-1.284856081009*xtemp+1.146428942680))
      expx3 = exp(-(0.677952528000*xeps+0.785129725933*xepsp-3.600407361984*xtemp-1.056580066681))
      expx4 = exp(-(-8.224933624268*xeps+0.102857425809*xepsp-1.170001864433*xtemp-0.834357798100))
      expx5 = exp(-(-0.262212038040*xeps-0.370604902506*xepsp+1.530593752861*xtemp-0.557993113995))
      ex

In [32]:
xeps = (eps -0.000000000000)/1.000000000000
xepsp = (log(epsp/1.000000000000) -0.000000000000)/10.819778284410
xtemp = (temp -20.000000000000)/480.000000000000
expx0 = exp(-(-0.171193018556*xeps-0.498235881329*xepsp+1.572309255600*xtemp-0.549710810184))
expx1 = exp(-(-0.182183891535*xeps-0.279642552137*xepsp+1.381547451019*xtemp-1.007667899132))
expx2 = exp(-(-0.121008850634*xeps-0.093622177839*xepsp-1.284856081009*xtemp+1.146428942680))
expx3 = exp(-(0.677952528000*xeps+0.785129725933*xepsp-3.600407361984*xtemp-1.056580066681))
expx4 = exp(-(-8.224933624268*xeps+0.102857425809*xepsp-1.170001864433*xtemp-0.834357798100))
expx5 = exp(-(-0.262212038040*xeps-0.370604902506*xepsp+1.530593752861*xtemp-0.557993113995))
expx6 = exp(-(0.363313168287*xeps+0.307944297791*xepsp-1.739808201790*xtemp-0.515264987946))
expx7 = exp(-(-0.171282619238*xeps-0.936020970345*xepsp-0.700896143913*xtemp-0.525115251541))
expx8 = exp(-(-2.934058904648*xeps+0.024969343096*xepsp-0.818501532078*xtemp+0.164255663753))
expx9 = exp(-(-0.298778355122*xeps-0.517931699753*xepsp-7.580852508545*xtemp-1.864312648773))
expx10 = exp(-(1.177027940750*xeps-0.308994054794*xepsp-1.684179663658*xtemp-0.859441757202))
expx11 = exp(-(-0.231339216232*xeps-0.290423154831*xepsp+1.502451419830*xtemp-1.198348641396))
expx12 = exp(-(235.784103393555*xeps-0.099293000996*xepsp+0.760096788406*xtemp+0.819592595100))
expx13 = exp(-(-23.344444274902*xeps+0.164729803801*xepsp-0.949119210243*xtemp-0.500462353230))
expx14 = exp(-(0.329279005527*xeps+0.054267201573*xepsp-1.555212616920*xtemp-0.580829739571))
exp20 = exp(0.000755758665*(1/(1+expx0))+0.348879784346*(1/(1+expx1))-1.622294902802*(1/(1+expx2))-1.744126558304*(1/(1+expx3))+1.189603447914*(1/(1+expx4))+0.773437738419*(1/(1+expx5))-1.077992796898*(1/(1+expx6))+0.195966541767*(1/(1+expx7))+0.371459305286*(1/(1+expx8))-2.175866603851*(1/(1+expx9))+0.267415910959*(1/(1+expx10))+0.117423757911*(1/(1+expx11))-1.837396621704*(1/(1+expx12))-1.631284832954*(1/(1+expx13))-0.915312469006*(1/(1+expx14))-0.220450609922)
exp21 = exp(-1.520736217499*(1/(1+expx0))-0.598750412464*(1/(1+expx1))-0.567928969860*(1/(1+expx2))+0.733987450600*(1/(1+expx3))-1.956816315651*(1/(1+expx4))-0.990542054176*(1/(1+expx5))-0.024108286947*(1/(1+expx6))-0.921862065792*(1/(1+expx7))-1.585801959038*(1/(1+expx8))-0.359389543533*(1/(1+expx9))-0.568457961082*(1/(1+expx10))-0.462055325508*(1/(1+expx11))+0.937371969223*(1/(1+expx12))+0.713293492794*(1/(1+expx13))-0.270313948393*(1/(1+expx14))-0.580874204636)
exp22 = exp(-0.263961493969*(1/(1+expx0))+0.340663641691*(1/(1+expx1))-1.086898207664*(1/(1+expx2))-0.190290063620*(1/(1+expx3))+1.526942133904*(1/(1+expx4))+0.134074971080*(1/(1+expx5))-0.458628684282*(1/(1+expx6))-0.210141628981*(1/(1+expx7))+0.690639853477*(1/(1+expx8))-0.559289455414*(1/(1+expx9))-0.561830639839*(1/(1+expx10))+0.010643188842*(1/(1+expx11))-2.779141187668*(1/(1+expx12))+2.203900575638*(1/(1+expx13))-0.874182760715*(1/(1+expx14))-0.327307760715)
exp23 = exp(-1.667145609856*(1/(1+expx0))-1.261272549629*(1/(1+expx1))-1.568691968918*(1/(1+expx2))-1.300865530968*(1/(1+expx3))-0.965722560883*(1/(1+expx4))-1.076323390007*(1/(1+expx5))-0.399949252605*(1/(1+expx6))-2.976247072220*(1/(1+expx7))-3.980113983154*(1/(1+expx8))-1.212576389313*(1/(1+expx9))-0.478132873774*(1/(1+expx10))-1.447541594505*(1/(1+expx11))-0.287288188934*(1/(1+expx12))-0.847129225731*(1/(1+expx13))-0.899507462978*(1/(1+expx14))-0.965191483498)
exp24 = exp(-0.421280384064*(1/(1+expx0))-0.564329922199*(1/(1+expx1))+1.009604096413*(1/(1+expx2))+0.335408806801*(1/(1+expx3))-0.515781700611*(1/(1+expx4))-0.189889833331*(1/(1+expx5))+0.561685442924*(1/(1+expx6))-0.274590790272*(1/(1+expx7))-0.640589475632*(1/(1+expx8))-0.454831272364*(1/(1+expx9))+0.185221791267*(1/(1+expx10))-1.204036474228*(1/(1+expx11))+2.715961456299*(1/(1+expx12))-2.415366411209*(1/(1+expx13))+0.508277475834*(1/(1+expx14))+0.001537762932)
exp25 = exp(-1.879916071892*(1/(1+expx0))-1.522165775299*(1/(1+expx1))-0.468759000301*(1/(1+expx2))+1.417994260788*(1/(1+expx3))+0.370010912418*(1/(1+expx4))-2.718472957611*(1/(1+expx5))+0.469961762428*(1/(1+expx6))-2.314279079437*(1/(1+expx7))-0.682480514050*(1/(1+expx8))-1.616387128830*(1/(1+expx9))+0.202725619078*(1/(1+expx10))-1.451031565666*(1/(1+expx11))+0.099444560707*(1/(1+expx12))-1.293660640717*(1/(1+expx13))+0.203722134233*(1/(1+expx14))-0.991643249989)
exp26 = exp(-0.589749693871*(1/(1+expx0))-0.571380496025*(1/(1+expx1))+0.120540380478*(1/(1+expx2))+0.763658463955*(1/(1+expx3))-1.608781814575*(1/(1+expx4))-0.913990020752*(1/(1+expx5))+0.719076871872*(1/(1+expx6))-1.206779360771*(1/(1+expx7))-0.747950553894*(1/(1+expx8))-0.689776837826*(1/(1+expx9))-0.001232341980*(1/(1+expx10))-0.592846393585*(1/(1+expx11))+1.381900072098*(1/(1+expx12))+0.327141195536*(1/(1+expx13))+0.125081762671*(1/(1+expx14))-0.179510131478)
p10 = -1.147175073624*(exp20/(1+exp20)**2)
p11 = 0.396453082561*(exp21/(1+exp21)**2)
p12 = -1.731730222702*(exp22/(1+exp22)**2)
p13 = 0.341490298510*(exp23/(1+exp23)**2)
p14 = 0.917140424252*(exp24/(1+exp24)**2)
p15 = 0.416720062494*(exp25/(1+exp25)**2)
p16 = 0.469032645226*(exp26/(1+exp26)**2)
p20 = expx0/(1+expx0)**2
p21 = expx1/(1+expx1)**2
p22 = expx2/(1+expx2)**2
p23 = expx3/(1+expx3)**2
p24 = expx4/(1+expx4)**2
p25 = expx5/(1+expx5)**2
p26 = expx6/(1+expx6)**2
p27 = expx7/(1+expx7)**2
p28 = expx8/(1+expx8)**2
p29 = expx9/(1+expx9)**2
p210 = expx10/(1+expx10)**2
p211 = expx11/(1+expx11)**2
p212 = expx12/(1+expx12)**2
p213 = expx13/(1+expx13)**2
p214 = expx14/(1+expx14)**2
xs0 = (0.000755758665*p10-1.520736217499*p11-0.263961493969*p12-1.667145609856*p13-0.421280384064*p14-1.879916071892*p15-0.589749693871*p16)*p20
xs1 = (0.348879784346*p10-0.598750412464*p11+0.340663641691*p12-1.261272549629*p13-0.564329922199*p14-1.522165775299*p15-0.571380496025*p16)*p21
xs2 = (-1.622294902802*p10-0.567928969860*p11-1.086898207664*p12-1.568691968918*p13+1.009604096413*p14-0.468759000301*p15+0.120540380478*p16)*p22
xs3 = (-1.744126558304*p10+0.733987450600*p11-0.190290063620*p12-1.300865530968*p13+0.335408806801*p14+1.417994260788*p15+0.763658463955*p16)*p23
xs4 = (1.189603447914*p10-1.956816315651*p11+1.526942133904*p12-0.965722560883*p13-0.515781700611*p14+0.370010912418*p15-1.608781814575*p16)*p24
xs5 = (0.773437738419*p10-0.990542054176*p11+0.134074971080*p12-1.076323390007*p13-0.189889833331*p14-2.718472957611*p15-0.913990020752*p16)*p25
xs6 = (-1.077992796898*p10-0.024108286947*p11-0.458628684282*p12-0.399949252605*p13+0.561685442924*p14+0.469961762428*p15+0.719076871872*p16)*p26
xs7 = (0.195966541767*p10-0.921862065792*p11-0.210141628981*p12-2.976247072220*p13-0.274590790272*p14-2.314279079437*p15-1.206779360771*p16)*p27
xs8 = (0.371459305286*p10-1.585801959038*p11+0.690639853477*p12-3.980113983154*p13-0.640589475632*p14-0.682480514050*p15-0.747950553894*p16)*p28
xs9 = (-2.175866603851*p10-0.359389543533*p11-0.559289455414*p12-1.212576389313*p13-0.454831272364*p14-1.616387128830*p15-0.689776837826*p16)*p29
xs10 = (0.267415910959*p10-0.568457961082*p11-0.561830639839*p12-0.478132873774*p13+0.185221791267*p14+0.202725619078*p15-0.001232341980*p16)*p210
xs11 = (0.117423757911*p10-0.462055325508*p11+0.010643188842*p12-1.447541594505*p13-1.204036474228*p14-1.451031565666*p15-0.592846393585*p16)*p211
xs12 = (-1.837396621704*p10+0.937371969223*p11-2.779141187668*p12-0.287288188934*p13+2.715961456299*p14+0.099444560707*p15+1.381900072098*p16)*p212
xs13 = (-1.631284832954*p10+0.713293492794*p11+2.203900575638*p12-0.847129225731*p13-2.415366411209*p14-1.293660640717*p15+0.327141195536*p16)*p213
xs14 = (-0.915312469006*p10-0.270313948393*p11-0.874182760715*p12-0.899507462978*p13+0.508277475834*p14+0.203722134233*p15+0.125081762671*p16)*p214
s0 = -0.171193018556*xs0-0.182183891535*xs1-0.121008850634*xs2+0.677952528000*xs3-8.224933624268*xs4-0.262212038040*xs5+0.363313168287*xs6-0.171282619238*xs7-2.934058904648*xs8-0.298778355122*xs9+1.177027940750*xs10-0.231339216232*xs11+235.784103393555*xs12-23.344444274902*xs13+0.329279005527*xs14
s1 = -0.498235881329*xs0-0.279642552137*xs1-0.093622177839*xs2+0.785129725933*xs3+0.102857425809*xs4-0.370604902506*xs5+0.307944297791*xs6-0.936020970345*xs7+0.024969343096*xs8-0.517931699753*xs9-0.308994054794*xs10-0.290423154831*xs11-0.099293000996*xs12+0.164729803801*xs13+0.054267201573*xs14
s2 = 1.572309255600*xs0+1.381547451019*xs1-1.284856081009*xs2-3.600407361984*xs3-1.170001864433*xs4+1.530593752861*xs5-1.739808201790*xs6-0.700896143913*xs7-0.818501532078*xs8-7.580852508545*xs9-1.684179663658*xs10+1.502451419830*xs11+0.760096788406*xs12-0.949119210243*xs13-1.555212616920*xs14
deps = s0 * 977.555715042962/1.000000000000
depsp = s1 * 977.555715042962/(epsp*10.819778284410)
dtemp = s2 * 977.555715042962/480.000000000000

In [33]:
deps,depsp,dtemp

(132.0722278659824, 0.0005599218637349831, -0.9256411664152612)

In [41]:
x=np.array([[xeps],[xepsp],[xtemp]])
x

array([[0.64999785],
       [0.90552119],
       [0.65628871]])

In [42]:
if ('tanh' in ANN_name) : s = tanhPrime2Layers(x)
elif ('sigmoid' in ANN_name) : s = sigmoidPrime2Layers(x)
else : s = 0
s

array([[ 0.13510455],
       [ 0.11148621],
       [-0.45450889]])

In [43]:
scaleOut = np.array([[rangeEntries[3]/rangeEntries[0]],[rangeEntries[3]/rangeEntries[1]],[rangeEntries[3]/rangeEntries[2]]])
scaleOut

array([[977.55571504],
       [ 90.34895996],
       [  2.03657441]])

In [44]:
scaled = s*scaleOut
scaled

array([[132.07222787],
       [ 10.07266339],
       [ -0.92564117]])

In [52]:
sigEpsNN = deps
sigEps = scaled[0][0]
print(sigEpsNN)
print('Ana : ',sigEps, abs((sigEpsNN-sigEps)/sigEps))

132.0722278659824
Ana :  132.07222786595148 2.3413567227521216e-13


In [53]:
scaled[1][0]/testData[0,1]

0.0005599218637344952

In [50]:
sigEpspNN = scaled[1][0]/testData[0,1]
print(sigEpspNN)
print('Num : ',sigEpspN, abs((sigEpspNN-sigEpspN)/sigEpspN))
print('Ana : ',sigEpsp, abs((sigEpspNN-sigEpsp)/sigEpsp))

0.0005599218637344952


NameError: name 'sigEpspN' is not defined

In [ ]:
sigTNN = scaled[2][0]
print(sigTNN)
print('Num : ',sigTN, abs((sigTNN-sigTN)/sigTN))
print('Ana : ',sigT, abs((sigTNN-sigT)/sigT))

In [ ]:
if ('tanh' in ANN_name) : sp = tanhPrime2Layers(inputTest.T)
elif ('sigmoid' in ANN_name) : sp = sigmoidPrime2Layers(inputTest.T)
else : sp = 0
ss = sp*scaleOut

In [ ]:
error = np.abs((ss[0,:] - testData[:,4]) / testData[:,4])
"EPS", error.max(), error.min(), error.mean()

In [ ]:
v1 = ss[0,:]
v2 = testData[:,4]
num = np.sum((v2-np.mean(v2))*(v1-np.mean(v1)))
den = math.sqrt(np.sum((v2-np.mean(v2))**2)*np.sum((v1-np.mean(v1))**2))
R = num/den
#np.mean(np.abs(((NNoutput+1)-(predictedOutput+1))/(NNoutput+1)))*100
plt.figure(figsize = (12, 8))
plt.rc('text', usetex = True)
plt.scatter(v2, v1, color='red', label="$AN\!N\ prediction$")
plt.plot([np.min(v2),np.max(v2)], [np.min(v2),np.max(v2)], color = 'black', label='$Best\ fit$')
plt.xlim(np.min(v2), np.max(v2))
plt.ylim(np.min(v2), np.max(v2))
plt.grid()
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Analytical\ \partial\sigma/\partial\varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Predicted\ \partial\sigma/\partial\varepsilon^{p}$', fontsize = 16)
plt.text(np.min(v2) + 0.2*(np.max(v2)-np.min(v2)), np.min(v2)+ 0.7*(np.max(v2)-np.min(v2)), '$R = %.8f$' % (R), fontsize=20)
#plt.title(r'$Map\ of\ errors\ \varepsilon^{p} / T$', fontsize = 20)
plt.savefig(ANN_name + 'R-sig-eps.svg')
plt.show() 

In [ ]:
levelsT = ['$> 5.00 \%$','$3.00 - 5.00 \%$','$1.00 - 3.00 \%$','$0.50 - 1.00 \%$','$< 0.50 \%$']
levelsC = [0.05, 0.03, 0.01, 0.005]

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], T_test[error >= levelsC[0]], color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\varepsilon^{p}(\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-eps-1.svg')
plt.show()
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\varepsilon^{p}(\varepsilon^{p},\dot\varepsilon^{p})$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-eps-2.svg')
plt.show()

In [ ]:
fig = plt.figure(figsize = (8, 8))
plt.rc('text', usetex = True)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), T_test[error >= levelsC[0]], color='black', label=levelsT[0])
ax.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
ax.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
ax.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
ax.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
ax.set_xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
ax.set_ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
ax.set_zlabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\varepsilon^{p}(\varepsilon^{p},\dot\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-eps-3D.svg')
plt.show()

In [ ]:
nnvals = ss[1]/testData[:,1]
error = np.abs((nnvals - testData[:,5]) / testData[:,5])
"EPSP", error.max(), error.min(), error.mean()

In [ ]:
v1 = ss[1]/testData[:,1]
v2 = testData[:,5]
num = np.sum((v2-np.mean(v2))*(v1-np.mean(v1)))
den = math.sqrt(np.sum((v2-np.mean(v2))**2)*np.sum((v1-np.mean(v1))**2))
R = num/den
#np.mean(np.abs(((NNoutput+1)-(predictedOutput+1))/(NNoutput+1)))*100
plt.figure(figsize = (12, 8))
plt.rc('text', usetex = True)
plt.scatter(v2, v1, color='red', label="$AN\!N\ prediction$")
plt.plot([np.min(v2),np.max(v2)], [np.min(v2),np.max(v2)], color = 'black', label='$Best\ fit$')
plt.xlim(np.min(v2), np.max(v2))
plt.ylim(np.min(v2), np.max(v2))
plt.grid()
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Analytical\ \partial\sigma/\partial\dot{\varepsilon}^{p}$', fontsize = 16)
plt.ylabel(r'$Predicted\ \partial\sigma/\partial\dot{\varepsilon}^{p}$', fontsize = 16)
plt.text(np.min(v2) + 0.2*(np.max(v2)-np.min(v2)), np.min(v2)+ 0.7*(np.max(v2)-np.min(v2)), '$R = %.8f$' % (R), fontsize=20)
plt.savefig(ANN_name + 'R-sig-epsp.svg')
plt.show() 

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], T_test[error >= levelsC[0]], color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\dot{\varepsilon}^{p}(\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-epsp-1.svg')
plt.show()
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\dot{\varepsilon}^{p}(\varepsilon^{p},\dot\varepsilon^{p})$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-epsp-2.svg')
plt.show()

In [ ]:
fig = plt.figure(figsize = (8, 8))
plt.rc('text', usetex = True)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), T_test[error >= levelsC[0]], color='black', label=levelsT[0])
ax.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
ax.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
ax.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
ax.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
ax.set_xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
ax.set_ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
ax.set_zlabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial\dot{\varepsilon}^{p}(\varepsilon^{p},\dot\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-epsp-3D.svg')
plt.show()

In [ ]:
nnvals = ss[2]
error = np.abs((nnvals - testData[:,6]) / testData[:,6])
"T", error.max(), error.min(), error.mean()

In [ ]:
v1 = ss[2]
v2 = testData[:,6]
num = np.sum((v2-np.mean(v2))*(v1-np.mean(v1)))
den = math.sqrt(np.sum((v2-np.mean(v2))**2)*np.sum((v1-np.mean(v1))**2))
R = num/den
#np.mean(np.abs(((NNoutput+1)-(predictedOutput+1))/(NNoutput+1)))*100
plt.figure(figsize = (12, 8))
plt.rc('text', usetex = True)
plt.scatter(v2, v1, color='red', label="$AN\!N\ prediction$")
plt.plot([np.min(v2),np.max(v2)], [np.min(v2),np.max(v2)], color = 'black', label='$Best\ fit$')
plt.xlim(np.min(v2), np.max(v2))
plt.ylim(np.min(v2), np.max(v2))
plt.grid()
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Analytical\ \partial\sigma/\partial T$', fontsize = 16)
plt.ylabel(r'$Predicted\ \partial\sigma/\partial T$', fontsize = 16)
plt.text(np.min(v2) + 0.2*(np.max(v2)-np.min(v2)), np.min(v2)+ 0.7*(np.max(v2)-np.min(v2)), '$R = %.8f$' % (R), fontsize=20)
#plt.title(r'$Map\ of\ errors\ \varepsilon^{p} / T$', fontsize = 20)
#if (saveFigures) : plt.savefig(dataPath + '/' + 'NN-' + model.name + '-error-sig.svg', bbox_inches = 'tight', pad_inches = 0)
plt.savefig(ANN_name + 'R-sig-T.svg')
plt.show() 

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], T_test[error >= levelsC[0]], color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial T(\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-T-1.svg')
plt.show()
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
plt.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), color='black', label=levelsT[0])
plt.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), color='red', label=levelsT[1])
plt.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), color='orange', label=levelsT[2])
plt.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), color='yellow', label=levelsT[3])
plt.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
plt.xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
plt.ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial T(\varepsilon^{p},\dot\varepsilon^{p})$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-T-2.svg')
plt.show()

In [ ]:
fig = plt.figure(figsize = (8, 8))
plt.rc('text', usetex = True)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(eps_test[error >= levelsC[0]], np.exp(epsp_test[error >= levelsC[0]]), T_test[error >= levelsC[0]], color='black', label=levelsT[0])
ax.scatter(eps_test[(error < levelsC[0]) & (error >= levelsC[1])], np.exp(epsp_test[(error < levelsC[0]) & (error >= levelsC[1])]), T_test[(error < levelsC[0]) & (error >= levelsC[1])], color='red', label=levelsT[1])
ax.scatter(eps_test[(error < levelsC[1]) & (error >= levelsC[2])], np.exp(epsp_test[(error < levelsC[1]) & (error >= levelsC[2])]), T_test[(error < levelsC[1]) & (error >= levelsC[2])], color='orange', label=levelsT[2])
ax.scatter(eps_test[(error < levelsC[2]) & (error >= levelsC[3])], np.exp(epsp_test[(error < levelsC[2]) & (error >= levelsC[3])]), T_test[(error < levelsC[2]) & (error >= levelsC[3])], color='yellow', label=levelsT[3])
ax.scatter(eps_test[error < levelsC[3]], np.exp(epsp_test[error < levelsC[3]]), T_test[error < levelsC[3]], color='green', label=levelsT[4])
plt.legend(loc = 'lower right',fancybox = True, numpoints = 1, fontsize = 14)
ax.set_xlabel(r'$Plastic\ strain\ \varepsilon^{p}$', fontsize = 16)
ax.set_ylabel(r'$Plastic\ strain\ rate\ \dot{\varepsilon^{p}}$', fontsize = 16)
ax.set_zlabel(r'$Temperature\ T^{\circ}C$', fontsize = 16)
plt.title(r'$Map\ of\ errors\ \partial\sigma/\partial T(\varepsilon^{p},\dot\varepsilon^{p},T)$', fontsize = 20)
plt.savefig(ANN_name + 'Map-derivative-T-3D.svg')
plt.show()

In [ ]:
TotalFrames = len(glob.glob(dataPath + '/' + ANN_name + '/ANN-*.npz'))
framesWide = 1000
emax = np.array([])
emin = np.array([])
emean = np.array([])
epmax = np.array([])
epmin = np.array([])
epmean = np.array([])
tmax = np.array([])
tmin = np.array([])
tmean = np.array([])
smax = np.array([])
smin = np.array([])
smean = np.array([])
for i in range(TotalFrames):
    nm = dataPath + '/' + ANN_name + '/ANN-' + str(i) + '.npz'
    NN = np.load(nm)
    w1 = NN['w1']
    w2 = NN['w2']
    w3 = NN['w3']
    b1 = NN['b1']
    b2 = NN['b2']
    b3 = NN['b3']
    minEntries = NN['minEntries']
    maxEntries = NN['maxEntries']
    rangeEntries = maxEntries - minEntries
    logBase = NN['logBase']
    if ('tanh' in ANN_name) : sp = tanhPrime2Layers(inputTest.T)
    elif ('sigmoid' in ANN_name) : sp = sigmoidPrime2Layers(inputTest.T)
    else : sp = 0
    ss = sp*scaleOut
    error = np.abs((ss[0,:] - testData[:,4]) / testData[:,4])
    emax=np.append(emax,error.max())
    emin=np.append(emin,error.min())
    emean=np.append(emean,error.mean())    
    error = np.abs((ss[1]/testData[:,1] - testData[:,5]) / testData[:,5])
    epmax=np.append(epmax,error.max())
    epmin=np.append(epmin,error.min())
    epmean=np.append(epmean,error.mean())    
    error = np.abs((ss[2] - testData[:,6]) / testData[:,6])
    tmax=np.append(tmax,error.max())
    tmin=np.append(tmin,error.min())
    tmean=np.append(tmean,error.mean())
    if ('tanh' in ANN_name) : sp = tanh2Layers(inputTest.T)
    elif ('sigmoid' in ANN_name) : sp = sigmoid2Layers(inputTest.T)
    else : sp = 0
    error = np.abs((sp[0,:] - testData[:,3]) / testData[:,3])
    smax=np.append(smax,error.max())
    smin=np.append(smin,error.min())
    smean=np.append(smean,error.mean())   

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
xx = np.linspace(1,TotalFrames*framesWide,TotalFrames)
plt.plot(xx,100*smax, label = r"$\sigma$",  linewidth = 3)
plt.plot(xx,100*emax, label = r"$\partial\sigma/\partial\varepsilon^p$",  linewidth = 3)
plt.plot(xx,100*epmax, label = r"$\partial\sigma/\partial\dot{\varepsilon^p}$",  linewidth = 3)
plt.plot(xx,100*tmax, label = r"$\partial\sigma/\partial T$",  linewidth = 3)
plt.grid()
plt.ylim(0, 10)
plt.xlabel(r'$Iteration$', fontsize = 16)
plt.ylabel(r'$Average\ absolute\ relative\ error\ $AARE$(\%)$', fontsize = 16)
plt.title(r'$Maximum\ error\ of\ the\ Neural\ Network\ on\ partial\ derivatives$', fontsize = 16)
plt.legend(loc = 'upper right',fancybox = True, numpoints = 1, fontsize = 14)
plt.savefig(ANN_name + 'max-error.svg')
plt.show()

In [ ]:
plt.figure(figsize = (12, 9))
plt.rc('text', usetex = True)
xx = np.linspace(1,TotalFrames*framesWide,TotalFrames)
plt.plot(xx,100*smean, label = r"$\sigma$",  linewidth = 3)
plt.plot(xx,100*emean, label = r"$\partial\sigma/\partial\varepsilon^p$",  linewidth = 3)
plt.plot(xx,100*epmean, label = r"$\partial\sigma/\partial\dot{\varepsilon^p}$",  linewidth = 3)
plt.plot(xx,100*tmean, label = r"$\partial\sigma/\partial T$",  linewidth = 3)
plt.grid()
plt.ylim(0, 5)
plt.xlabel(r'$Iteration$', fontsize = 16)
plt.ylabel(r'$Average\ absolute\ relative\ error\ $AARE$(\%)$', fontsize = 16)
plt.title(r'$Mean\ error\ of\ the\ Neural\ Network\ on\ partial\ derivatives$', fontsize = 16)
plt.legend(loc = 'upper right',fancybox = True, numpoints = 1, fontsize = 14)
plt.savefig(ANN_name + 'mean-error.svg')
plt.show()

In [ ]:
file_object = open('compare.txt', 'a')
file_object.write(ANN_name+'\n')
file_object.write('smean: %5.3f \n' %(100*np.mean(smean[-6:-1])))
file_object.write('emean: %5.3f \n' %(100*np.mean(emean[-6:-1])))
file_object.write('epmean: %5.3f \n' %(100*np.mean(epmean[-6:-1])))
file_object.write('tmean: %5.3f \n' %(100*np.mean(tmean[-6:-1])))
file_object.write('smax: %5.3f \n' %(100*np.mean(smax[-6:-1])))
file_object.write('emax: %5.3f \n' %(100*np.mean(emax[-6:-1])))
file_object.write('epmax: %5.3f \n' %(100*np.mean(epmax[-6:-1])))
file_object.write('tmax: %5.3f \n' %(100*np.mean(tmax[-6:-1])))
file_object.close()

In [ ]:
file_object = open('compare.csv', 'a')
file_object.write(ANN_name+';')
file_object.write('%5.3f;' %(100*np.mean(smean[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(emean[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(epmean[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(tmean[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(smax[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(emax[-6:-1])))
file_object.write('%5.3f;' %(100*np.mean(epmax[-6:-1])))
file_object.write('%5.3f\n' %(100*np.mean(tmax[-6:-1])))
file_object.close()